In [61]:
import sys
import gsi_float_api
import numpy as np
import csv
from datetime import datetime
import h5py

### Allocate Boards

Allocate boards on the correct localhost, don't proceed until status is "OK"

In [62]:
gsi_conf = gsi_float_api.GSIConfig()
print(gsi_conf)
gsi = gsi_float_api.Float_GSI(conf=gsi_conf)

In [63]:
 #gsi.delete_allocation('74d79426-feae-11ea-be85-0242ac110014')

'{"error": "Allocation id 74d79426-feae-11ea-be85-0242ac110014 invalid"}'

In [64]:
alloc_id, status = gsi.request_allocation(3) # All 3 work

print("Allocation ID: " + str(alloc_id))
print("Status: " + str(status))

Allocation ID: 752d36b8-ff5a-11ea-bc79-0242ac110014
Status: OK


### Load Data

Loading dataset from server since it is mounted there, loading query set from local npy file

In [65]:
#This can take 30 minutes
dataset_id, result = gsi.import_dataset("/efs/data/qa/any_vision/deep-image-96.npy")
print("Dataset ID: " + str(dataset_id))
print("Status: " + str(result))

Dataset ID: b6df79ec-ff5d-11ea-bc79-0242ac110014
Status: OK


In [66]:
#create the queryset
fl = np.load("/efs/data/qa/any_vision/deep1b_queries.npy")

#testing query's working
#search = gsi.Search()

#fp = np.load('/efs/data/qa/any_vision/queries_1000.npy')
#status = gsi.load_data(alloc_id, dataset_id, typical_nqueries=1000, max_nqueries=1000)
#distance, indices, search_time, status = search.knn_composite_cosine_by_queries(alloc_id, dataset_id, fp,10)

In [60]:
#gsi.unload_data(alloc_id,dataset_id)

'{"error": "Allocation id f6117874-ff4c-11ea-be85-0242ac110014 invalid"}'

### Search

Search using different Hamming values

In [67]:
search = gsi.Search()

ham = 10
time_list = []

total_search_time = 0
print("Search Times (QPS) for GSI_APU faiss float32 Search:")
print("----------------------------------------------")
print()
print("RUNNING ...")

timed = []
neigh = []
dist = []

status = gsi.load_data(alloc_id, dataset_id, typical_nqueries=1, max_nqueries=1)
print(status)
total_search = 0
for i in range(len(fl)):
    distance, indices, search_time, status = search.knn_composite_cosine_by_queries(alloc_id, dataset_id, [np.arange(96).tolist(),fl[i].tolist()],10)
    
    timed.append(search_time)
    neigh.append(indices)
    dist.append(distance)
    
    if i % 1000 == 0:
        print(i)
    
    total_search = total_search + search_time

print("----------------------------------------------")
print("Total Search Time: " +str(total_search))
print("Hamming_k Value of: " + str(ham),end = " runs at ")
print(str(np.round(1.0/(total_search/10000),2)) + " Average Queries per Second")

Search Times (QPS) for GSI_APU faiss float32 Search:
----------------------------------------------

RUNNING ...
OK


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
attrs = {
        "algo": "float32_1board",
        "batch_mode": False,
        "best_search_time": total_search/10000,
        "candidates": 1,
        "expect_extra": False,
        "name": "float32",
        "run_count": 2,
        "distance": "angular",
        "count": 10,
        'dataset': 'nytimes-256-angular'
    }

In [ ]:
#SAVE IN ANN_BENCH FORMAT
fn = "apu_float32_1_1"    
f = h5py.File(fn, 'w')

for k, v in attrs.items():
        f.attrs[k] = v

times = f.create_dataset('times', (len(timed),), 'f')
    
neighbors = f.create_dataset('neighbors', (len(neigh), 10), 'i')
    
distances = f.create_dataset('distances', (len(dist), 10), 'f')

times = timed
neighbors = neigh
distances = dist
             
f.close()

In [31]:
status = gsi.unload_data(alloc_id, dataset_id)
status = gsi.delete_allocation(alloc_id)

## TESTING FORMAT, not relevent

In [25]:
check = h5py.File("angular_hnsw_M_12_efConstruction_800_post_0_false_1","r")
check.keys()

<KeysViewHDF5 ['distances', 'metrics', 'neighbors', 'times']>

In [26]:
list(check[list(check.keys())[1]])[:100]

['knn']